Pour essayer les exemples présents dans ce tutoriel : 

::: {.cell .markdown}

In [ ]:
#| output: asis
#| include: true
#| eval: true
import sys
sys.path.insert(1, '../../../../') #insert the utils module
from utils import print_badges
#print_badges(__file__)
print_badges("content/course/manipulation/02a_pandas_tutorial.qmd")

:::

Le _package_ `pandas` est l'une des briques centrales de l'écosystème de
la data-science. Son créateur, Wes McKinney, l'a pensé comme 
une surcouche à la librairie `numpy` pour introduire le _DataFrame_
qui est un objet central dans des langages comme `R` 
ou `Stata` mais qui était absent dans `Python`. `pandas` est rapidement
devenu un incontournable de la _data-science_. L'ouvrage
de référence de @mckinney2012python présente de manière plus 
ample ce _package_.

Ce tutoriel vise à introduire aux concepts
de base de ce package par l'exemple et à introduire à certaines
des tâches les plus fréquentes de (re)structuration
des données du _data-scientist_. Il ne s'agit pas d'un ensemble
exhaustif de commandes: `pandas` est un package tentaculaire 
qui permet de réaliser la même opération de nombreuses manières. 
Nous nous concentrerons ainsi sur les éléments les plus pertinents
dans le cadre d'une introduction à la _data-science_ et laisserons
les utilisateurs intéressés approfondir leurs connaissances
dans les ressources foisonnantes qu'il existe sur le sujet. 

Dans ce tutoriel `pandas`, nous allons utiliser:

* Les émissions de gaz à effet de serre estimées au niveau communal par l'ADEME. Le jeu de données est 
disponible sur [data.gouv](https://www.data.gouv.fr/fr/datasets/inventaire-de-gaz-a-effet-de-serre-territorialise/#_)
et requêtable directement dans `Python` avec
[cet url](https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert)

Le [chapitre suivant](#pandasTP) permettra de mettre en application des éléments présents dans ce chapitre avec
les données ci-dessus associées à des données de contexte au niveau communal<a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1).


::: {.cell .markdown}

```{=html}
<a name="cite_note-1"></a>1. [^](#cite_ref-1)
```

 Idéalement, on utiliserait les données
[disponibles sur le site de l'Insee](https://www.insee.fr/fr/statistiques/3560121) mais celles-ci nécessitent un peu de travail
de nettoyage qui n'entre pas dans le cadre de ce TP. 
Pour faciliter l'import de données `Insee`, il est recommandé d'utiliser le package
[`pynsee`](https://github.com/InseeFrLab/Py-Insee-Data) qui simplifie l'accès aux données
de l'Insee disponibles sur le site web [insee.fr](https://www.insee.fr/fr/accueil)
ou via des API. 
:::

Pour simplifier l'accès aux données du site [insee.fr](https://www.insee.fr/fr/accueil)
nous allons utiliser un package nommé `pynsee`. 
Son code source est disponible sur 
[`Github`](https://github.com/InseeFrLab/Py-Insee-Data).
Pour l'installer depuis la cellule d'un `Notebook Jupyter`:


In [ ]:
#| eval: false
!pip install pynsee

```{=html}
</div>
```

:::


⚠️ `pandas` offre la possibilité d'importer des données
directement depuis un url. C'est l'option prise dans ce tutoriel.
Si vous préfèrez, pour des
raisons d'accès au réseau ou de performance, importer depuis un poste local,
vous pouvez télécharger les données et changer
les commandes d'import avec le chemin adéquat plutôt que l'url. 


Nous suivrons les conventions habituelles dans l'import des packages


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynsee.download

Pour obtenir des résultats reproductibles, on peut fixer la racine du générateur
pseudo-aléatoire. 


In [ ]:
np.random.seed(123)

Au cours de cette démonstration des principales fonctionalités de `pandas`, et
lors du chapitre suivant,
je recommande de se référer régulièrement aux ressources suivantes:

* L'[aide officielle de pandas](https://pandas.pydata.org/docs/user_guide/index.html).
Notamment, la
[page de comparaison des langages](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/index.html)
est très utile
* La _cheatsheet suivante_, [issue de ce post](https://becominghuman.ai/cheat-sheets-for-ai-neural-networks-machine-learning-deep-learning-big-data-678c51b4b463)

![Cheasheet pandas](https://cdn-images-1.medium.com/max/2000/1*YhTbz8b8Svi22wNVvqzneg.jpeg)

# Logique de pandas

L'objet central dans la logique `pandas` est le `DataFrame`.
Il s'agit d'une structure particulière de données
à deux dimensions, structurées en alignant des lignes et colonnes. Les colonnes
peuvent être de type différent.

Un `DataFrame` est composé des éléments suivants:

* l'indice de la ligne ;
* le nom de la colonne ;
* la valeur de la donnée ;

Structuration d'un DataFrame pandas,
empruntée à <https://medium.com/epfl-extension-school/selecting-data-from-a-pandas-dataframe-53917dc39953>:


In [ ]:
import shutil
import requests

url = 'https://miro.medium.com/max/700/1*6p6nF4_5XpHgcrYRrLYVAw.png'
response = requests.get(url, stream=True)
with open('featured.png', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

![](featured.png)

Le concept de *tidy* data, popularisé par Hadley Wickham via ses packages `R`,
est parfaitement pertinent pour décrire la structure d'un `DataFrame pandas`.
Les trois règles sont les suivantes:

* Chaque variable possède sa propre colonne ;
* Chaque observation possède sa propre ligne ;
* Une valeur, matérialisant une observation d'une variable,
se trouve sur une unique cellule.


![Concept de tidy data (emprunté à H. Wickham)](https://d33wubrfki0l68.cloudfront.net/6f1ddb544fc5c69a2478e444ab8112fb0eea23f8/91adc/images/tidy-1.png)


::: {.cell .markdown}

```{=html}
<div class="alert alert-warning" role="alert" style="color: rgba(0,0,0,.8); background-color: white; margin-top: 1em; margin-bottom: 1em; margin:1.5625emauto; padding:0 .6rem .8rem!important;overflow:hidden; page-break-inside:avoid; border-radius:.25rem; box-shadow:0 .2rem .5rem rgba(0,0,0,.05),0 0 .05rem rgba(0,0,0,.1); transition:color .25s,background-color .25s,border-color .25s ; border-right: 1px solid #dee2e6 ; border-top: 1px solid #dee2e6 ; border-bottom: 1px solid #dee2e6 ; border-left:.2rem solid #ffc10780;">
<h3 class="alert-heading"><i class="fa fa-lightbulb-o"></i> Hint</h3>
```


Les `DataFrames` sont assez rapides en `Python`<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2) et permettent de traiter en local de manière efficace des tables de
données comportant plusieurs millions d'observations (en fonction de la configuration de l'ordinateur)
et dont la volumétrie peut être conséquente (plusieurs centaines
de Mo). Néanmoins,  passé un certain seuil, qui dépend de la puissance de la machine mais aussi de la complexité
de l'opération effectuée, le DataFrame `pandas` peut montrer certaines limites. Dans ce cas, il existe différentes
solutions: `Dask` (dataframe aux opérations parallélisés), `SQL` (notamment `Postgres`),
`Spark` (solution _big data_). Un chapitre spécial de ce cours est consacré à `Dask`.


```{=html}
</div>
```

:::



::: {.cell .markdown}

```{=html}
<a name="cite_note-2"></a>2. [^](#cite_ref-2)
```

  En `R`, les deux formes de dataframes qui se sont imposées récemment sont les `tibbles` (package `dplyr`)
et les `data.tables` (package `data.table`). `dplyr` reprend la syntaxe `SQL` de manière relativement
transparente ce qui rend la syntaxe très proche de celle de `pandas`. Cependant,
alors que `dplyr` supporte très mal les données dont la volumétrie dépasse 1Go, `pandas` s'en
accomode bien. Les performances de `pandas` sont plus proches de celles de `data.table`, qui est
connu pour être une approche efficace avec des données de taille importante.
:::

Concernant la syntaxe, une partie des commandes python est inspirée par la logique `SQL`. On retrouvera ainsi
des instructions relativement transparentes.

Il est vivement recommandé, avant de se lancer dans l'écriture d'une
fonction, de se poser la question de son implémentation native dans `numpy`, `pandas`, etc.
En particulier, la plupart du temps, s'il existe une solution implémentée dans une librairie, il convient
de l'utiliser.

# Les Series

En fait, un DataFrame est une collection d'objets appelés `pandas.Series`.
Ces `Series` sont des objets d'une dimension qui sont des extensions des
array-unidimensionnels `numpy`. En particulier, pour faciliter le traitement
de données catégorielles ou temporelles, des types de variables
supplémentaires sont disponibles dans `pandas` par rapport à
`numpy` (`categorical`, `datetime64` et `timedelta64`). Ces
types sont associés à des méthodes optimisées pour faciliter le traitement
de ces données.

Il ne faut pas négliger l'attribut `dtype` d'un objet
`pandas.Series` car cela a une influence déterminante sur les méthodes
et fonctions pouvant être utilisées (on ne fait pas les mêmes opérations
sur une donnée temporelle et une donnée catégorielle) et le volume en
mémoire d'une variable (le type de la variable détermine le volume
d'information stocké pour chaque élément ; être trop précis est parfois
néfaste).

Il existe plusieurs types possibles pour un `pandas.Series`.
Le type `object` correspond aux types Python `str` ou `mixed`.
Il existe un type particulier pour les variables dont le nombre de valeurs
est une liste finie et relativement courte, le type `category`.
Il faut bien examiner les types de son DataFrame, et convertir éventuellement
les types lors de l'étape de `data cleaning`.

## Indexation

La différence essentielle entre une `Series` et un objet `numpy` est l'indexation.
Dans `numpy`,
l'indexation est implicite ; elle permet d'accéder à une donnée (celle à
l'index situé à la position *i*).
Avec une `Series`, on peut bien-sûr utiliser un indice de position mais on peut
surtout faire appel à des indices plus explicites.
Par exemple,


In [ ]:
taille = pd.Series(
    [1.,1.5,1],
    index = ['chat', 'chien', 'koala']
)

taille.head()

Cette indexation permet d'accéder à des valeurs de la `Series`
via une valeur de l'indice. Par
exemple, `taille['koala']`:


In [ ]:
taille['koala']

L'existence d'indice rend le *subsetting* particulièrement aisé, ce que vous
pouvez expérimenter dans les TP

::: {.cell .markdown}

In [ ]:
#| output: 'asis'
#| include: true
#| eval: true

import sys
sys.path.insert(1, '../../../../') #insert the utils module
from utils import print_badges

#print_badges(__file__)
print_badges("content/course/manipulation/02b_pandas_TP.qmd")

:::



Pour transformer un objet `pandas.Series` en array `numpy`,
on utilise la méthode `values`. Par exemple, `taille.values`:


In [ ]:
taille.values

Un avantage des `Series` par rapport à un *array* `numpy` est que
les opérations sur les `Series` alignent
automatiquement les données à partir des labels.
Avec des `Series` labélisées, il n'est ainsi pas nécessaire
de se poser la question de l'ordre des lignes.
L'exemple dans la partie suivante permettra de s'en assurer.


## Valeurs manquantes

Par défaut, les valeurs manquantes sont affichées `NaN` et sont de type `np.nan` (pour
les valeurs temporelles, i.e. de type `datatime64`, les valeurs manquantes sont
`NaT`).

On a un comportement cohérent d'agrégation lorsqu'on combine deux `DataFrames` (ou deux colonnes).
Par exemple,


In [ ]:
x = pd.DataFrame(
    {'prix': np.random.uniform(size = 5),
     'quantite': [i+1 for i in range(5)]
    },
    index = ['yaourt','pates','riz','tomates','gateaux']
)
x

In [ ]:
y = pd.DataFrame(
    {'prix': [np.nan, 0, 1, 2, 3],
     'quantite': [i+1 for i in range(5)]
    },
    index = ['tomates','yaourt','gateaux','pates','riz']
)
y

In [ ]:
x + y

donne bien une valeur manquante pour la ligne `tomates`. Au passage, on peut remarquer que l'agrégation
a tenu compte des index.

Il est possible de supprimer les valeurs manquantes grâce à `dropna()`.
Cette méthode va supprimer toutes les lignes où il y a au moins une valeur manquante.
Il est aussi possible de supprimer seulement les colonnes où il y a des valeurs manquantes
dans un DataFrame avec `dropna()` avec le paramètre `axis=1` (par défaut égal à 0).

Il est également possible de remplir les valeurs manquantes grâce à la méthode `fillna()`.

# Le DataFrame pandas

Le `DataFrame` est l'objet central de la librairie `pandas`.
Il s'agit d'une collection de `pandas.Series` (colonnes) alignées par les index.
Les types des variables peuvent différer.

Un `DataFrame` non-indexé a la structure suivante:


In [ ]:
df = pd.DataFrame(
    {'taille': [1.,1.5,1],
    'poids' : [3, 5, 2.5]
    },
    index = ['chat', 'chien', 'koala']
)
df.reset_index()

Alors que le même `DataFrame` indexé aura la structure suivante:


In [ ]:
df = pd.DataFrame(
    {'taille': [1.,1.5,1],
    'poids' : [3, 5, 2.5]
    },
    index = ['chat', 'chien', 'koala']
)
df.head()

## Les attributs et méthodes utiles

Pour présenter les méthodes les plus pratiques pour l'analyse de données,
on peut partir de l'exemple des consommations de CO2 communales issues
des données de l'Ademe. Cette base de données est exploitée plus intensément
dans le TP.

::: {.cell .markdown}

In [ ]:
#| output: 'asis'
#| include: true
#| eval: true

import sys
sys.path.insert(1, '../../../../') #insert the utils module
from utils import print_badges

#print_badges(__file__)
print_badges("content/course/manipulation/02b_pandas_TP.qmd")

:::

L'import de données depuis un fichier plat se fait avec la fonction [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html):


In [ ]:
df = pd.read_csv("https://koumoul.com/s/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert")
df

::: {.cell .markdown}

```{=html}
<div class="alert alert-info" role="alert" style="color: rgba(0,0,0,.8); background-color: white; margin-top: 1em; margin-bottom: 1em; margin:1.5625emauto; padding:0 .6rem .8rem!important;overflow:hidden; page-break-inside:avoid; border-radius:.25rem; box-shadow:0 .2rem .5rem rgba(0,0,0,.05),0 0 .05rem rgba(0,0,0,.1); transition:color .25s,background-color .25s,border-color .25s ; border-right: 1px solid #dee2e6 ; border-top: 1px solid #dee2e6 ; border-bottom: 1px solid #dee2e6 ; border-left:.2rem solid #007bff80;">
<h3 class="alert-heading"><i class="fa fa-comment"></i> Note</h3>
```



Dans un processus de production, où normalement on connait les types des variables du `DataFrame` qu'on va importer,
il convient de préciser les types avec lesquels on souhaite importer les données
(argument `dtype`, sous la forme d'un dictionnaire).
Cela est particulièrement important lorsqu'on désire utiliser une colonne
comme une variable textuelle mais qu'elle comporte des attributs proches d'un nombre
qui vont inciter `pandas` à l'importer sous forme de variable numérique.

Par exemple, une colonne `[00001,00002,...] ` risque d'être importée comme une variable numérique, ignorant l'information des premiers 0 (qui peuvent pourtant la distinguer de la séquence 1, 2, etc.). Pour s'assurer que `pandas` importe sous forme textuelle la variable, on peut utiliser `dtype = {"code": "str"}`
Sinon, on peut importer le csv, et modifier les types avec `astype()`.
Avec `astype`, on peut gérer les erreurs de conversion avec le paramètre `errors`.


```{=html}
</div>
```

:::

L'affichage des DataFrames est très ergonomique. On obtiendrait le même *output*
avec `display(df)`<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3). Les premières et dernières lignes s'affichent
automatiquement. Autrement, on peut aussi faire:

* `head` qui permet, comme son
nom l'indique, de n'afficher que les premières lignes ;
* `tail` qui permet, comme son
nom l'indique, de n'afficher que les dernières lignes
* `sample` qui permet d'afficher un échantillon aléatoire de *n* lignes.
Cette méthode propose de nombreuses options.

::: {.cell .markdown}

```{=html}
<a name="cite_note-3"></a>3. [^](#cite_ref-3)
```

 Il est préférable d'utiliser la fonction `display` (ou tout simplement
taper le nom du `DataFrame`) qu'utiliser la fonction `print`. Le
`display` des objets `pandas` est assez esthétique, contrairement à `print`
qui renvoie du texte brut.
:::

::: {.cell .markdown}

```{=html}
<div class="alert alert-danger" role="alert" style="color: rgba(0,0,0,.8); background-color: white; margin-top: 1em; margin-bottom: 1em; margin:1.5625emauto; padding:0 .6rem .8rem!important;overflow:hidden; page-break-inside:avoid; border-radius:.25rem; box-shadow:0 .2rem .5rem rgba(0,0,0,.05),0 0 .05rem rgba(0,0,0,.1); transition:color .25s,background-color .25s,border-color .25s ; border-right: 1px solid #dee2e6 ; border-top: 1px solid #dee2e6 ; border-bottom: 1px solid #dee2e6 ; border-left: .2rem solid #ff0039;">
<i class="fa fa-exclamation-triangle"></i> Warning</h3>
```


Il faut faire attention au `display` et aux
commandes qui révèlent des données (`head`, `tail`, etc.)
dans un `Notebook` ou un `Markdown` qui exploite
des données confidentielles lorsqu'on utilise `Git`.

En effet, on peut se
retrouver à partager des données, involontairement, dans l'historique
`Git`. Avec un `R Markdown`, il suffit d'ajouter les sorties au fichier
`.gitignore` (par exemple avec une balise de type `*.html`). Avec un
`Notebook Jupyter`, la démarche est plus compliquée car les fichiers
`.ipynb` intègrent dans le même document, texte, sorties et mise en forme.

Techniquement, il est possible d'appliquer des filtres avec `Git`
(voir
[ici](http://timstaley.co.uk/posts/making-git-and-jupyter-notebooks-play-nice/))
mais c'est une démarche très complexe. 
Ce post de l'équipe à l'origine de [nbdev2](https://www.fast.ai/posts/2022-08-25-jupyter-git.html)
résume bien le problème du contrôle de version avec `Git` et des solutions qui
peuvent y être apportées.

Une solution est d'utiliser [`Quarto`](https://quarto.org/) qui permet de générer les
`.ipynb` en _output_ d'un document texte, ce qui facilite le contrôle sur les 
éléments présents dans le document.


```{=html}
</div>
```

:::



### Dimensions et structure du DataFrame

Les premières méthodes utiles permettent d'afficher quelques
attributs d'un `DataFrame`.


In [ ]:
df.axes

In [ ]:
df.columns

In [ ]:
df.index

Pour connaître les dimensions d'un DataFrame, on peut utiliser quelques méthodes
pratiques:


In [ ]:
df.ndim

In [ ]:
df.shape

In [ ]:
df.size

Pour déterminer le nombre de valeurs uniques d'une variable, plutôt que chercher à écrire soi-même une fonction,
on utilise la
méthode `nunique`. Par exemple,


In [ ]:
df['Commune'].nunique()

`pandas` propose énormément de méthodes utiles. 
Voici un premier résumé, accompagné d'un comparatif avec `R`

| Opération                     | pandas       | dplyr (`R`)    | data.table (`R`)           |
|-------------------------------|--------------|----------------|----------------------------|
| Récupérer le nom des colonnes | `df.columns` | `colnames(df)` | `colnames(df)`             |
| Récupérer les indices<a name="cite_ref-4"></a>[<sup>[4]</sup>](#cite_note-4)     | `df.index`   |                |`unique(df[,get(key(df))])` |
| Récupérer les dimensions      | `df.shape` | `c(nrow(df), ncol(df))` | `c(nrow(df), ncol(df))` |
| Récupérer le nombre de valeurs uniques d'une variable | `df['myvar'].nunique()` | `df %>%  summarise(distinct(myvar))` | `df[,uniqueN(myvar)]` |

::: {.cell .markdown}

```{=html}
<a name="cite_note-4"></a>4. [^](#cite_ref-4)
```

 Le principe d'indice n'existe pas dans `dplyr`. Ce qui s'approche le plus des indices, au sens de
`pandas`, sont les *clés* en `data.table`.
:::

### Statistiques agrégées

`pandas` propose une série de méthodes pour faire des statistiques
agrégées de manière efficace.

On peut, par exemple, appliquer des méthodes pour compter le nombre de lignes,
faire une moyenne ou une somme de l'ensemble des lignes


In [ ]:
df.count()

In [ ]:
df.mean(numeric_only = True)

In [ ]:
df.sum(numeric_only = True)

In [ ]:
df.nunique()

In [ ]:
df.quantile(q = [0.1,0.25,0.5,0.75,0.9], numeric_only = True)

::: {.cell .markdown}

```{=html}
<div class="alert alert-danger" role="alert" style="color: rgba(0,0,0,.8); background-color: white; margin-top: 1em; margin-bottom: 1em; margin:1.5625emauto; padding:0 .6rem .8rem!important;overflow:hidden; page-break-inside:avoid; border-radius:.25rem; box-shadow:0 .2rem .5rem rgba(0,0,0,.05),0 0 .05rem rgba(0,0,0,.1); transition:color .25s,background-color .25s,border-color .25s ; border-right: 1px solid #dee2e6 ; border-top: 1px solid #dee2e6 ; border-bottom: 1px solid #dee2e6 ; border-left: .2rem solid #ff0039;">
<i class="fa fa-exclamation-triangle"></i> Warning</h3>
```


La version 2.0 de `Pandas` a introduit un changement
de comportement dans les méthodes d'agrégation. 

Il est dorénavant nécessaire de préciser quand on désire
effectuer des opérations si on désire ou non le faire
exclusivement sur les colonnes numériques. C'est pour cette 
raison qu'on exlicite ici l'argument `numeric_only = True`. 
Ce comportement
était par le passé implicite. 


```{=html}
</div>
```

:::


Il faut toujours regarder les options de ces fonctions en termes de valeurs manquantes, car
ces options sont déterminantes dans le résultat obtenu.

Les exercices de TD visent à démontrer l'intérêt de ces méthodes dans quelques cas précis.

::: {.cell .markdown}

In [ ]:
#| output: 'asis'
#| include: true
#| eval: true

import sys
sys.path.insert(1, '../../../../') #insert the utils module
from utils import print_badges

#print_badges(__file__)
print_badges("content/course/manipulation/02b_pandas_TP.qmd")

:::



Le tableau suivant récapitule le code équivalent pour avoir des
statistiques sur toutes les colonnes d'un dataframe en `R`.


| Opération                     | pandas       | dplyr (`R`)    | data.table (`R`)           |
|-------------------------------|--------------|----------------|----------------------------|
| Nombre de valeurs non manquantes | `df.count()`   | `df %>% summarise_each(funs(sum(!is.na(.))))` | `df[, lapply(.SD, function(x) sum(!is.na(x)))]`
| Moyenne de toutes les variables | `df.mean()` | `df %>% summarise_each(funs(mean((., na.rm = TRUE))))` | `df[,lapply(.SD, function(x) mean(x, na.rm = TRUE))]`| TO BE CONTINUED |

La méthode `describe` permet de sortir un tableau de statistiques
agrégées:


In [ ]:
df.describe()

### Méthodes relatives aux valeurs manquantes

Les méthodes relatives aux valeurs manquantes peuvent être mobilisées
en conjonction des méthodes de statistiques agrégées. C'est utiles lorsqu'on
désire obtenir une idée de la part de valeurs manquantes dans un jeu de
données


In [ ]:
df.isnull().sum()

On trouvera aussi la référence à `isna()` qui est la même méthode que `isnull()`.

# Graphiques rapides

Les méthodes par défaut de graphique
(approfondies dans la [partie visualisation](#visualisation))
sont pratiques pour
produire rapidement un graphique, notamment après des opérations
complexes de maniement de données.

En effet, on peut appliquer la méthode `plot()` directement à une `pandas.Series`:


In [ ]:
#| eval: false

df['Déchets'].plot()
df['Déchets'].hist()
df['Déchets'].plot(kind = 'hist', logy = True)

In [ ]:
plt.figure()
fig = df['Déchets'].plot()
fig
#plt.savefig('plot_base.png', bbox_inches='tight')

plt.figure()
fig = df['Déchets'].hist()
fig
#plt.savefig('plot_hist.png', bbox_inches='tight')

plt.figure()
fig = df['Déchets'].plot(kind = 'hist', logy = True)
fig
#plt.show()
#plt.savefig('plot_hist_log.png', bbox_inches='tight')

La sortie est un objet `matplotlib`. La *customisation* de ces
figures est ainsi
possible (et même désirable car les graphiques `matplotlib`
sont, par défaut, assez rudimentaires), nous en verrons quelques exemples.


# Accéder à des éléments d'un DataFrame

## Sélectionner des colonnes

En SQL, effectuer des opérations sur les colonnes se fait avec la commande
`SELECT`. Avec `pandas`,
pour accéder à une colonne dans son ensemble on peut
utiliser plusieurs approches:

* `dataframe.variable`, par exemple `df.Energie`.
Cette méthode requiert néanmoins d'avoir des
noms de colonnes sans espace.
* `dataframe[['variable']]` pour renvoyer la variable sous
forme de `DataFrame` ou `dataframe['variable']` pour
la renvoyer sous forme de `Series`. Par exemple, `df[['Autres transports']]`
ou `df['Autres transports']`. C'est une manière préférable de procéder.

## Accéder à des lignes

Pour accéder à une ou plusieurs valeurs d'un `DataFrame`,
il existe deux manières conseillées de procéder, selon la
forme des indices de lignes ou colonnes utilisés:

* `df.loc`: utilise les labels
* `df.iloc`: utilise les indices

::: {.cell .markdown}

```{=html}
<div class="alert alert-danger" role="alert" style="color: rgba(0,0,0,.8); background-color: white; margin-top: 1em; margin-bottom: 1em; margin:1.5625emauto; padding:0 .6rem .8rem!important;overflow:hidden; page-break-inside:avoid; border-radius:.25rem; box-shadow:0 .2rem .5rem rgba(0,0,0,.05),0 0 .05rem rgba(0,0,0,.1); transition:color .25s,background-color .25s,border-color .25s ; border-right: 1px solid #dee2e6 ; border-top: 1px solid #dee2e6 ; border-bottom: 1px solid #dee2e6 ; border-left: .2rem solid #ff0039;">
<i class="fa fa-exclamation-triangle"></i> Warning</h3>
```


Les bouts de code utilisant la structure `df.ix`
sont à bannir car la fonction est *deprecated* et peut
ainsi disparaître à tout moment.


```{=html}
</div>
```

:::

`iloc` va se référer à l'indexation de 0 à *N* où *N* est égal à `df.shape[0]` d'un
`pandas.DataFrame`. `loc` va se référer aux valeurs de l'index
de `df`.

Par exemple, avec le `pandas.DataFrame` `df_example`:


In [ ]:
df_example = pd.DataFrame(
    {'year': [2012, 2014, 2013, 2014], 'sale': [55, 40, 84, 31]})
df_example

- `df_example.loc[1, :]` donnera la première ligne de `df` (ligne où l'indice `month` est égal à 1) ;
- `df_example.iloc[1, :]` donnera la deuxième ligne (puisque l'indexation en `Python` commence à 0) ;
- `df_example.iloc[:, 1]` donnera la deuxième colonne, suivant le même principe.




# Principales manipulation de données

L'objectif du [TP pandas](#pandasTP) est de se familiariser plus avec ces
commandes à travers l'exemple des données des émissions de C02.

Les opérations les plus fréquentes en SQL `sont` résumées par le tableau suivant.
Il est utile de les connaître (beaucoup de syntaxes de maniement de données
reprennent ces termes) car, d'une
manière ou d'une autre, elles couvrent la plupart
des usages de manipulation des données

| Opération | SQL | pandas | dplyr (`R`) | data.table (`R`) |
|-----|-----------|--------|-------------|------------------|
| Sélectionner des variables par leur nom | `SELECT` | `df[['Autres transports','Energie']]` | `df %>% select(Autres transports, Energie)` | `df[, c('Autres transports','Energie')]` |
| Sélectionner des observations selon une ou plusieurs conditions; | `FILTER` | `df[df['Agriculture']>2000]` | `df %>% filter(Agriculture>2000)` | `df[Agriculture>2000]` |
| Trier la table selon une ou plusieurs variables | `SORT BY` | `df.sort_values(['Commune','Agriculture'])` | `df %>% arrange(Commune, Agriculture)` | `df[order(Commune, Agriculture)]` |
| Ajouter des variables qui sont fonction d’autres variables; | `SELECT *, LOG(Agriculture) AS x FROM df` | `df['x'] = np.log(df['Agriculture'])`  |  `df %>% mutate(x = log(Agriculture))` | `df[,x := log(Agriculture)]` |
| Effectuer une opération par groupe | `GROUP BY` | `df.groupby('Commune').mean()` | `df %>% group_by(Commune) %>% summarise(m = mean)` | `df[,mean(Commune), by = Commune]` |
| Joindre deux bases de données (*inner join*) | `SELECT * FROM table1 INNER JOIN table2 ON table1.id = table2.x` | `table1.merge(table2, left_on = 'id', right_on = 'x')` | `table1 %>% inner_join(table2, by = c('id'='x'))` | `merge(table1, table2, by.x = 'id', by.y = 'x')` |

## Opérations sur les colonnes: `select`, `mutate`, `drop`

Les `DataFrames` pandas sont des objets *mutables* en langage `Python`,
c'est-à-dire qu'il est possible de faire évoluer le `DataFrame` au grès
des opérations. L'opération la plus classique consiste à ajouter ou retirer
des variables à la table de données.


In [ ]:
df_new = df.copy()

::: {.cell .markdown}

```{=html}
<div class="alert alert-danger" role="alert" style="color: rgba(0,0,0,.8); background-color: white; margin-top: 1em; margin-bottom: 1em; margin:1.5625emauto; padding:0 .6rem .8rem!important;overflow:hidden; page-break-inside:avoid; border-radius:.25rem; box-shadow:0 .2rem .5rem rgba(0,0,0,.05),0 0 .05rem rgba(0,0,0,.1); transition:color .25s,background-color .25s,border-color .25s ; border-right: 1px solid #dee2e6 ; border-top: 1px solid #dee2e6 ; border-bottom: 1px solid #dee2e6 ; border-left: .2rem solid #ff0039;">
<i class="fa fa-exclamation-triangle"></i> Warning</h3>
```


Attention au comportement de `pandas` lorsqu'on crée une duplication
d'un `DataFrame`.

Par défaut, `pandas` effectue une copie par référence. Dans ce
cas, les deux objets (la copie et l'objet copié) restent reliés. Les colonnes
crées sur l'un vont être répercutées sur l'autre. Ce comportement permet de
limiter l'inflation en mémoire de `Python`. En faisant ça, le deuxième
objet prend le même espace mémoire que le premier. Le package `data.table`
en `R` adopte le même comportement, contrairement à `dplyr`.

Cela peut amener à quelques surprises si ce comportement d'optimisation
n'est pas anticipé. Si vous voulez, par sécurité, conserver intact le
premier DataFrame, faites appel à une copie profonde (*deep copy*) en
utilisant la méthode `copy`, comme ci-dessus.

Attention toutefois, cela a un coût mémoire.
Avec des données volumineuses, c'est une pratique à utiliser avec précaution.


```{=html}
</div>
```

:::



La manière la plus simple d'opérer pour ajouter des colonnes est
d'utiliser la réassignation. Par exemple, pour créer une variable
`x` qui est le `log` de la
variable `Agriculture`:


In [ ]:
df_new['x'] = np.log(df_new['Agriculture'])

Il est possible d'appliquer cette approche sur plusieurs colonnes. Un des
intérêts de cette approche est qu'elle permet de recycler le nom de colonnes.


In [ ]:
vars = ['Agriculture', 'Déchets', 'Energie']

df_new[[v + "_log" for v in vars]] = np.log(df_new[vars])
df_new

Il est également possible d'utiliser la méthode `assign`. Pour des opérations
vectorisées, comme le sont les opérateurs de `numpy`, cela n'a pas d'intérêt.

Cela permet notamment d'enchainer les opérations sur un même `DataFrame` (notamment grâce au `pipe` que
nous verrons plus loin).
Cette approche utilise généralement
des *lambda functions*. Par exemple le code précédent (celui concernant une
seule variable) prendrait la forme:


In [ ]:
df_new.assign(Energie_log = lambda x: np.log(x['Energie']))

Dans les méthodes suivantes, il est possible de modifier le `pandas.DataFrame`
*en place*, c'est à dire en ne le réassignant pas, avec le paramètre `inplace = True`.
Par défaut, `inplace` est égal à `False` et pour modifier le `pandas.DataFrame`,
il convient de le réassigner.

On peut facilement renommer des variables avec la méthode `rename` qui
fonctionne bien avec des dictionnaires (pour renommer des colonnes il faut
préciser le paramètre `axis = 1`):


In [ ]:
df_new = df_new.rename({"Energie": "eneg", "Agriculture": "agr"}, axis=1)

Enfin, pour effacer des colonnes, on utilise la méthode `drop` avec l'argument
`columns`:


In [ ]:
df_new = df_new.drop(columns = ["eneg", "agr"])

## Réordonner

La méthode `sort_values` permet de réordonner un `DataFrame`. Par exemple,
si on désire classer par ordre décroissant de consommation de CO2 du secteur
résidentiel, on fera


In [ ]:
df = df.sort_values("Résidentiel", ascending = False)

Ainsi, en une ligne de code, on identifie les villes où le secteur
résidentiel consomme le plus.


## Filtrer

L'opération de sélection de lignes s'appelle `FILTER` en SQL. Elle s'utilise
en fonction d'une condition logique (clause `WHERE`). On sélectionne les
données sur une condition logique. Il existe plusieurs méthodes en `pandas`.

La plus simple est d'utiliser les *boolean mask*, déjà vus dans le chapitre
[`numpy`](#numpy).

Par exemple, pour sélectionner les communes dans les Hauts-de-Seine, on
peut utiliser le résultat de la méthode `str.startswith` (qui renvoie
`True` ou `False`) directement dans les crochets:


In [ ]:
df[df['INSEE commune'].str.startswith("92")].head(2)

Pour remplacer des valeurs spécifiques, on utilise la méthode `where` ou une
réassignation couplée à la méthode précédente.

Par exemple, pour assigner des valeurs manquantes aux départements du 92,
on peut faire cela


In [ ]:
df_copy = df.copy()
df_copy = df_copy.where(~df['INSEE commune'].str.startswith("92"))

et vérifier les résultats:


In [ ]:
df_copy[df['INSEE commune'].str.startswith("92")].head(2)
df_copy[~df['INSEE commune'].str.startswith("92")].head(2)

ou alors utiliser une réassignation plus classique:


In [ ]:
df_copy = df.copy()
df_copy[df_copy['INSEE commune'].str.startswith("92")] = np.nan

Il est conseillé de filtrer avec `loc` en utilisant un masque.
En effet, contrairement à `df[mask]`, `df.loc[mask, :]` permet d'indiquer clairement
à Python que l'on souhaite appliquer le masque aux labels de l'index.
Ce n'est pas le cas avec `df[mask]`.
D'ailleurs, lorsqu'on utilise la syntaxe `df[mask]`, `pandas` renvoie généralement un *warning*

## Opérations par groupe

En `SQL`, il est très simple de découper des données pour
effectuer des opérations sur des blocs cohérents et recollecter des résultats
dans la dimension appropriée.
La logique sous-jacente est celle du *split-apply-combine* qui est repris
par les langages de manipulation de données, auxquels `pandas`
[ne fait pas exception](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

L'image suivante, issue de
[ce site](https://unlhcc.github.io/r-novice-gapminder/16-plyr/)
représente bien la manière dont fonctionne l'approche
`split`-`apply`-`combine`

![Split-apply-combine](https://unlhcc.github.io/r-novice-gapminder/fig/12-plyr-fig1.png)


Ce [tutoriel](https://realpython.com/pandas-groupby/) sur le sujet
est particulièrement utile.

Pour donner quelques exemples, on peut créer une variable départementale qui
servira de critère de groupe.


In [ ]:
df['dep'] = df['INSEE commune'].str[:2]

En `pandas`, on utilise `groupby` pour découper les données selon un ou
plusieurs axes. Techniquement, cette opération consiste à créer une association
entre des labels (valeurs des variables de groupe) et des
observations.

Par exemple, pour compter le nombre de communes par département en SQL, on
utiliserait la requête suivante:

```sql
SELECT dep, count(INSEE commune)
FROM df
GROUP BY dep;
```

Ce qui, en `pandas`, donne:


In [ ]:
df.groupby('dep')["INSEE commune"].count()

La syntaxe est quasiment transparente. On peut bien-sûr effectuer des opérations
par groupe sur plusieurs colonnes. Par exemple,


In [ ]:
df.groupby('dep').mean(numeric_only = True)

A noter que la variable de groupe, ici `dep`, devient, par défaut, l'index
du DataFrame de sortie. Si on avait utilisé plusieurs variables de groupe,
on obtiendrait un objet multi-indexé. Sur la gestion des `multi-index`, on
pourra se référer à l'ouvrage `Modern Pandas` dont la référence est
donnée en fin de cours.

Tant qu'on n'appelle pas une action sur un `DataFrame` par groupe, du type
`head` ou `display`, `pandas` n'effectue aucune opération. On parle de
*lazy evaluation*. Par exemple, le résultat de `df.groupby('dep')` est
une transformation qui n'est pas encore évaluée:


In [ ]:
df.groupby('dep')

Il est possible d'appliquer plus d'une opération à la fois grâce à la méthode
`agg`. Par exemple, pour obtenir à la fois le minimum, la médiane et le maximum
de chaque département, on peut faire:


In [ ]:
numeric_columns = df.select_dtypes(['number']).columns
df.loc[:, numeric_columns.tolist() + ["dep"] ].groupby('dep').agg(['min',"median","max"], numeric_only = True)

La première ligne est présente pour nous faciliter la récupération des noms de colonnes des variables
numériques

## Appliquer des fonctions

`pandas` est, comme on a pu le voir, un package très flexible, qui
propose une grande variété de méthodes optimisées. Cependant, il est fréquent
d'avoir besoin de méthodes non implémentées.

Dans ce cas, on recourt souvent aux `lambda` functions. Par exemple, si
on désire connaître les communes dont le nom fait plus de 40 caractères,
on peut appliquer la fonction `len` de manière itérative:


In [ ]:
# Noms de communes superieurs à 40 caracteres
df[df['Commune'].apply(lambda s: len(s)>40)]

Cependant, toutes les `lambda` functions ne se justifient pas.
Par exemple, prenons
le résultat d'agrégation précédent. Imaginons qu'on désire avoir les résultats
en milliers de tonnes. Dans ce cas, le premier réflexe est d'utiliser
la `lambda` function suivante:


In [ ]:
numeric_columns = df.select_dtypes(['number']).columns
(df
    .loc[:, numeric_columns.tolist() + ["dep"] ]
    .groupby('dep')
    .agg(['min',"median","max"])
    .apply(lambda s: s/1000)
)

En effet, cela effectue le résultat désiré. Cependant, il y a mieux: utiliser
la méthode `div`:


In [ ]:
#| eval: false
import timeit
df_numeric = df.loc[:, numeric_columns.tolist() + ["dep"] ]
%timeit df_numeric.groupby('dep').agg(['min',"median","max"]).div(1000)
%timeit df_numeric.groupby('dep').agg(['min',"median","max"]).apply(lambda s: s/1000)

La méthode `div` est en moyenne plus rapide et a un temps d'exécution
moins variable. Dans ce cas, on pourrait même utiliser le principe
du *broadcasting* de numpy (cf. [chapitre numpy](numpy)) qui offre
des performances équivalentes:


In [ ]:
#| eval: false
%timeit df_numeric.groupby('dep').agg(['min',"median","max"])/1000

`apply` est plus rapide qu'une boucle (en interne, `apply` utilise `Cython`
pour itérer) mais reste moins rapide qu'une solution vectorisée quand
elle existe. Ce [site](https://realpython.com/fast-flexible-pandas/#pandas-apply)
propose des solutions, par exemple les méthodes `isin` ou `digitize`, pour
éviter de manuellement créer des boucles lentes.

En particulier, il faut noter que `apply` avec le paramètre `axis=1` est en générale lente.

## Joindre


Il est commun de devoir combiner des données issues de sources différentes.
Nous allons ici nous focaliser sur le cas le plus favorable qui est la situation
où une information permet d'apparier de manière exacte deux bases de données (autrement nous
serions dans une situation, beaucoup plus complexe, d'appariement flou<a name="cite_ref-6"></a>[<sup>[6]</sup>](#cite_note-6)).


```{=html}
<a name="cite_note-6"></a>6. [^](#cite_ref-6)
```

 Sur l'appariement flou, se reporter aux chapitres présentant `ElasticSearch`. 

La situation typique est l'appariement entre deux sources de données selon un identifiant
individuel. Ici, il s'agit d'un identifiant de code commune.

Il est recommandé de lire [ce guide assez complet sur la question des jointures avec R](https://www.book.utilitr.org/jointures.html)
qui donne des recommandations également utiles pour un utilisateur de `Python`.

![](https://pics.me.me/thumb_left-join-right-join-inner-join-full-outer-join-imgflip-com-66845242.png)

On utilise de manière indifférente les termes *merge* ou *join*.
Le deuxième terme provient de la syntaxe SQL.
En `Pandas`, dans la plupart des cas, on peut utiliser indifféremment `df.join` et `df.merge`

![](pandas_join.png)

Il est aussi possible de réaliser un merge en utilisant la fonction `pandas.concat()` avec `axis=1`.
Se référer à la documentation de `concat` pour voir les options possibles.

## Reshape

On présente généralement deux types de données:

* format __wide__: les données comportent des observations répétées, pour un même individu (ou groupe), dans des colonnes différentes
* format __long__: les données comportent des observations répétées, pour un même individu, dans des lignes différentes avec une colonne permettant de distinguer les niveaux d'observations

Un exemple de la distinction entre les deux peut être emprunté à l'ouvrage de référence d'Hadley Wickham, *R for Data Science*:

![](https://d33wubrfki0l68.cloudfront.net/3aea19108d39606bbe49981acda07696c0c7fcd8/2de65/images/tidy-9.png)


L'aide mémoire suivante aidera à se rappeler les fonctions à appliquer si besoin:

![](reshape.png)

Le fait de passer d'un format *wide* au format *long* (ou vice-versa) peut être extrêmement pratique car
certaines fonctions sont plus adéquates sur une forme de données ou sur l'autre.
En règle générale, avec `Python` comme avec `R`, les formats *long* sont souvent préférables.

Le chapitre suivant, qui fait office de TP, proposera des applications de ces principes

::: {.cell .markdown}

In [ ]:
#| output: 'asis'
#| include: true
#| eval: true

import sys
sys.path.insert(1, '../../../../') #insert the utils module
from utils import print_badges

#print_badges(__file__)
print_badges("content/course/manipulation/02b_pandas_TP.qmd")

:::

## Les pipe

En général, dans un projet, le nettoyage de données va consister en un ensemble de
méthodes appliquées à un `pandas.DataFrame`.
On a vu que `assign` permettait de créer une variable dans un `DataFrame`.
Il est également possible d'appliquer une fonction, appelée par exemple `my_udf` au
DataFrame grâce à `pipe`:

```python
df = (pd.read_csv(path2data)
            .pipe(my_udf))
```

L'utilisation des `pipe` rend le code très lisible et peut être très
pratique lorsqu'on enchaine des opérations sur le même
_dataset_. 

# Quelques enjeux de performance

La librairie `Dask` intègre la structure de `numpy`, `pandas` et `sklearn`.
Elle a vocation à traiter de données en grande dimension, ainsi elle ne sera pas
optimale pour des données qui tiennent très bien en RAM.
Il s'agit d'une librairie construite sur la parallélisation.
[Un chapitre dans ce cours](/dask.html) lui est consacré
Pour aller plus loin, se référer à la [documentation de `Dask`](https://docs.dask.org/en/latest/).


# Références

* Le site
[pandas.pydata](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html)
fait office de référence

* Le livre `Modern Pandas` de Tom Augspurger: https://tomaugspurger.github.io/modern-1-intro.html

::: {#refs}
:::

# Annexe {#annexe}

::: {.cell .markdown}

```{=html}
<details><summary>Téléchargement de <code>pynsee</code> avec barre de progrès 👇</summary>
```


Ce code vient du [package `pynsee`](https://github.com/InseeFrLab/Py-Insee-Data/blob/master/pynsee/download/__init__.py)...


In [ ]:
#| eval: false
import warnings
import os
import requests
import zipfile
from pathlib import Path
from shutil import copyfile, copyfileobj

# import tqdm.auto as tqdma
from tqdm import tqdm
from tqdm.utils import CallbackIOWrapper


def download_pb(url: str, fname: str, total: int = None):
    """Useful function to get request with a progress bar
    Borrowed from https://gist.github.com/yanqd0/c13ed29e29432e3cf3e7c38467f42f51
    Arguments:
        url {str} -- URL for the source file
        fname {str} -- Destination where data will be written
    """
    resp = requests.get(url, stream=True)

    if total is None:
        total = int(resp.headers.get('content-length', 0))

    with open(fname, 'wb') as file, tqdm(
            desc='Downloading: ',
            total=total,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)


def unzip_pb(fzip, dest, desc="Extracting"):
    """
    Useful function to unzip with progress bar
    Args:
        fzip: Filename of the zipped file
        dest: Destination where data must be written
        desc: Argument inherited from zipfile.ZipFile
    Returns:
        zipfile.Zipfile(fzip).extractall(dest) with progress
    """

    dest = Path(dest).expanduser()
    Path(dest).mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(fzip) as zipf, tqdm(
            desc=desc, unit="B", unit_scale=True, unit_divisor=1024,
            total=sum(getattr(i, "file_size", 0) for i in zipf.infolist()),
    ) as pbar:
        for i in zipf.infolist():
            if not getattr(i, "file_size", 0):  # directory
                zipf.extract(i, os.fspath(dest))
            else:
                with zipf.open(i) as fi, open(os.fspath(dest / i.filename), "wb") as fo:
                    copyfileobj(CallbackIOWrapper(pbar.update, fi), fo)


download_pb('https://github.com/InseeFrLab/Py-Insee-Data/archive/refs/heads/master.zip', 'pynsee.zip')

unzip_pb("pynsee.zip", "pynsee")

```{=html}
</details>
```

:::